<h1 style="color:#00A6D6;">Week 5: Distributing keys</h1>

Welcome to the new Julia sheet! As usual, we will ask you to use Julia to answer a few exercises. Most importantly, however, the purpose of these Julia sheets is for you to play around and build intuition by exploring and calculating things we do NOT ask you :-) We hope that you take advantage of using Julia this way.

* <a href="#Enc">Encoder</a>
* <a href="#Dec">Decoder</a>
* <a href="#All">Putting all together</a>

The include command that follows will include all the functions that you have used on the previous weeks including D for trace distance and minEntropy for computing the min-entropy. Please use them as you see fit to answer the exercises.

In [ ]:
include("source/main.jl")

This week we will implement a toy information reconciliation from scratch. It will be very similar to the scheme described in the videos except that we will use different codes. Let us first recall the communication scenario of a one-way information reconciliation protocol: 

<img src="source/oneway.png">

Alice holds a string $X_A$, Bob holds a string $X_B=X_A+S$ and we call $S$ the error vector. In order to help Bob recover $X_A$, Alice will encode her string in $C_A=\textrm{Enc}(X_A)$ and send $C_A$ to Bob via a public authentic channel. Bob will input $C_A$ and $X_B$ into a decoder that will produce an estimate of Alice's string $\hat X_A=\textrm{Dec}(C_A,X_B)$.

<a id=Enc></a>
<h2 style="color:#00A6D6;">Encoder</h2>

Let us first take a look at the encoder function. Given some parity check matrix $H$, the encoder function that we will use will encode $X_A$ into the syndrome of $X_A.$ That is: $C_A=H\cdot X_A^T$. 

So, let us write the encoding function which is simply a small function for computing syndromes:

In [ ]:
function getSyndrome(H,v)
    # Input:
    # ------
    # H - k x n parity check matrix
    # v - length n vector
    #
    # Output:
    # -------
    # s - syndrome of v with parity check matrix H
    return # your code
end

Note: The function getSyndrome(H,v) will be called later by other functions. In order to make it compatible with these other functions, the output should be a **row** vector.

<h3 style="color:#00A6D6;"> edX Exercise 1</h3>

Consider the parity check matrices:

In [ ]:
H_Golay = [1 0 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0;
 1 0 1 0 1 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0;
 1 0 1 1 0 1 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0;
 1 0 1 1 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0;
 1 1 0 0 1 1 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0;
 1 1 0 1 0 1 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0;
 1 1 0 1 1 0 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0;
 1 1 1 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0;
 1 1 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0;
 1 1 1 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0;
 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1];

H_Hamming = [1 0 1 0 1 0 1;
             0 1 1 0 0 1 1;
             0 0 0 1 1 1 1];

Let us investigate the codes induced by these matrices:

* How many different syndromes has the Hamming code?
* How many different syndromes has the Golay code?

Recall that the weight of a vector is the number of non-zero entries in the vector. Let $V_n^w$ be the set of binary vectors of length $n$ and weight $w$:

* What is the number of elements in $V_7^0$,$V_7^1$,$V_7^2$ and $V_7^3$?
* What is the number of elements in $V_{23}^0$,$V_{23}^1$,$V_{23}^2$ and $V_{23}^3$?

Now, compute the encodings of the following strings:

In [ ]:
v0 = [1 0 1 1 0 1 1]; # with the Hamming parity check matrix
v1 = [1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0]; # with the Golay parity check matrix

<a id=Dec></a>
<h2 style="color:#00A6D6;">Decoder</h2>


Now, let us take a look at the decoder. The protocol that we described in the lectures (known as syndrome decoding) works as follows: 
* Bob computes the syndrome of his string $C_B=H\cdot X_B^T$
* The sum of $C_A$ and $C_B$ is fed into an estimator module that will output an estimate of the error string $\hat S$.
* The decoder outputs $X_B+\hat S$

<img src="source/decoder.png">

We gave two parity check matrices in Exercise 1: $H_{\textrm{Hamming}}$ and $H_{\textrm{Golay}}$. They induce a unique encoder. However, there are many possible decoders that we could associate with them. If we look at the estimator module, it takes a syndrome and outputs an error estimate. We can choose the map between syndrome and outputs in many different ways. 

We are going to implement the following estimator map for the Hamming code:
* The zero syndrome gets mapped to the zero error string
* For each vector $v\in V^1_7$ compute the syndrome $s=H_{\textrm{Hamming}}\cdot v$ and assign the syndrome $s$ the error string $v$.

We have implemented this functionality as a Julia dictionary.

In [ ]:
dict_Hamming = Dict()
z = 0*eVecInt(7,1)'
s = 0*eVecInt(3,1)'
dict_Hamming[s] = z
for i=1:7
    vi = eVecInt(7,i)'
    s = getSyndrome(H_Hamming,vi)
    dict_Hamming[s] = vi
end

Now, if we want to know the output of the estimator for some concrete syndrome we would use the dictionary as follows:

In [ ]:
s = [0 1 1]
dict_Hamming[s]

Now you can implement the following estimator map for the Golay code:

* The zero syndrome gets mapped to the zero error string
* For each vector $v\in V^1_{23}$ compute the syndrome $s=H_{\textrm{Golay}}\cdot v$ and assign the syndrome $s$ the error string $v$.
* For each vector $v\in V^2_{23}$ compute the syndrome $s=H_{\textrm{Golay}}\cdot v$ and assign the syndrome $s$ the error string $v$.
* For each vector $v\in V^3_{23}$ compute the syndrome $s=H_{\textrm{Golay}}\cdot v$ and assign the syndrome $s$ the error string $v$.

In [ ]:
dict_Golay = Dict()

<h3 style="color:#00A6D6;"> edX Exercise 2</h3>

Compute the output of the estimator module of the Golay code with input error syndrome:

In [ ]:
s0 = [1 1 1 0 0 0 0 1 1 1 0]
s1 = [1 0 1 0 0 1 1 1 0 1 0]

<h3 style="color:#00A6D6;"> edX Exercise 3</h3>

Now we can complete the decoder. Compute the output of the decoder for the Golay code with the following inputs.

In [ ]:
# 1) 

Ca1 = [0 1 0 1 0 0 0 1 1 0 1]
Xb1 = [0 1 1 0 1 1 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0]

# 2)

Ca2 = [0 0 1 0 1 1 0 1 1 1 1]
Xb2 = [0 1 0 1 1 1 0 1 1 0 0 0 1 1 1 0 0 0 1 0 1 1 0]

In [ ]:
function decoder(Ca,Xb,estimatorDict,H)
    # Here goes your code
end

<a id=All></a>
<h2 style="color:#00A6D6;">Putting all together</h2>

Finally, we have all the ingredients for running together the encoder and the decoder. 
<h3 style="color:#00A6D6;"> edX Exercise 4</h3>

Given the following $X_A$ and $X_B$, what are the values of all the intermediate strings and what is the final decoder output?

In [ ]:
Xa3 = [0 1 1 0 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1 1 0]
Xb3 = [0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 0 1 1 0]